In [25]:
############################
# Archive
############################
# ToDo: remove \xa0 from article_text
# ToDo: check if words before " is a " appear again aftterwards in the article_text and remove them
# TODO H3: latLngToCell; cellToChildren/cellToParent; polygonToCells; cellAreaKm2 / getHexagonAreaAvgKm2; compactCells


# def calc_area(id):
#     # Overpass API URL
#     OVERPASS_API_URL = "https://overpass-api.de/api/interpreter"
#
#     # Build Overpass API query to fetch the relation and its member ways
#     overpass_query = f"""
#     [out:json][timeout:25];
#     relation({id});
#     way(r);
#     out geom;
#     """
#
#     # Fetch data from Overpass API
#     response = requests.post(OVERPASS_API_URL, data={"data": overpass_query})
#     if response.status_code != 200:
#         return ""
#         # raise RuntimeError(f"Overpass API request failed with status code {response.status_code}")
#     data = response.json()
#
#     # Check if any elements were returned
#     if not data['elements']:
#         return ""
#         # raise ValueError("No data found for the given relation ID.")
#
#     try:
#         # Extract all way geometries and combine them into a multipolygon
#         polygons = []
#         for element in data['elements']:
#             if element['type'] == 'way' and 'geometry' in element:
#                 coordinates = [(node['lon'], node['lat']) for node in element['geometry']]
#                 polygon = Polygon(coordinates)
#                 if not polygon.is_valid:
#                     # Repair invalid polygons
#                     polygon = make_valid(polygon)
#                 polygons.append(polygon)
#
#         # Combine the polygons into a multipolygon or union them into a single polygon
#         combined_polygon = unary_union(polygons)
#     except Exception as e:
#         return ""
#
#
#     # Ensure the result is valid
#     if not combined_polygon.is_valid:
#         return ""
#         # raise ValueError("The combined polygon is not valid even after repair!")
#
#     # Project the polygon to a planar coordinate system (EPSG:3857 - Web Mercator)
#     transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
#
#     def project_coords(geometry):
#         if geometry.geom_type == 'Polygon':
#             return Polygon([transformer.transform(*coord) for coord in geometry.exterior.coords])
#         elif geometry.geom_type == 'MultiPolygon':
#             return MultiPolygon([Polygon([transformer.transform(*coord) for coord in poly.exterior.coords]) for poly in geometry.geoms])
#         else:
#             return ""
#             # raise ValueError("Unsupported geometry type!")
#
#
#     projected_polygon = project_coords(combined_polygon)
#
#     # Calculate the area in square kilometers
#     area_km2 = projected_polygon.area / 1e6  # Convert square meters to square kilometers
#     return area_km2, combined_polygon

In [26]:
import json
import requests
from shapely.geometry import Polygon
import folium
from shapely.ops import transform
import pyproj
import h3
from agent_components.memory.long_term_memory import LongTermMemory
from agent_components.llms.chatAI import ChatAIHandler
import re
import pandas as pd
import os

In [27]:
with open(r"C:\Users\kaimo\Uni\Master\5_WS24_25-MA\geoparse-natural-disasters-with-llms\data\processed_GeoCoDe_test.json", "r", encoding="utf-8") as f:
    geocode_test_set = json.load(f)

In [28]:
geocode_test_set

[{'article_id': '9326279',
  'article_text': 'The Gulf of Oman or Sea of Oman (Arabic: خَلِيج عُمَان\u200e khalīj ʿumān; Persian: دریای عمان\u200e daryâ-ye omân) is a gulf that connects the Arabian Sea with the Strait of Hormuz, which then runs to the Persian Gulf. It borders Iran and Pakistan on the north, Oman on the south, and the United Arab Emirates on the west.',
  'toponyms': ['Arabian Sea',
   'Strait of Hormuz',
   'Persian Gulf',
   'Iran',
   'Pakistan',
   'Oman',
   'United Arab Emirates'],
  'toponym_data': [{'osm_id': '9328393',
    'coordinates': {'latitude': 12.436798, 'longitude': 62.073997},
    'toponym': 'Arabian Sea'},
   {'osm_id': '9326284',
    'coordinates': {'latitude': 26.4496678, 'longitude': 56.2798177},
    'toponym': 'Strait of Hormuz'},
   {'osm_id': '9326283',
    'coordinates': {'latitude': 27.2380905, 'longitude': 51.7498926},
    'toponym': 'Persian Gulf'},
   {'osm_id': '304938',
    'coordinates': {'latitude': 32.3088854, 'longitude': 53.6825806},

In [29]:
geo_relatable_harsh = [
    article for article in geocode_test_set
    if (any(word in article["article_text"] for word in ["lies", "located"]) and
        " is a " in article["article_text"] and
        any(word in article["article_text"] for word in ["km", "kilometer", "kilometre"]))
        #and not "area" in article["article_text"])
]
print(len(geo_relatable_harsh))
geo_relatable_harsh

204


[{'article_id': '1953975',
  'article_text': "Leningrad Nuclear Power Plant (Russian: Ленинградская атомная электростанция; Ленинградская АЭС (pronunciation\xa0(help·info))) is a nuclear power plant located in the town of Sosnovy Bor in Russia's Leningrad Oblast, on the southern shore of the Gulf of Finland, some 70 kilometres (43\xa0mi) to the west of the city centre of Saint Petersburg.",
  'toponyms': ["Russia's",
   'Leningrad Oblast',
   'Gulf of Finland',
   'Saint Petersburg'],
  'toponym_data': [{'osm_id': '60189',
    'coordinates': {'latitude': 61.62186, 'longitude': -74.7863859},
    'toponym': "Russia's"},
   {'osm_id': '176095',
    'coordinates': {'latitude': 59.8732948, 'longitude': 31.070656},
    'toponym': 'Leningrad Oblast'},
   {'osm_id': '9408222',
    'coordinates': {'latitude': 59.8737466, 'longitude': 26.3763389},
    'toponym': 'Gulf of Finland'},
   {'osm_id': '337422',
    'coordinates': {'latitude': 59.9393101, 'longitude': 30.0926255},
    'toponym': 'Saint

In [31]:
with open(
        r"C:\Users\kaimo\Uni\Master\5_WS24_25-MA\geoparse-natural-disasters-with-llms\output\reflective_candidate_resolution\fatal_error_and_invalid_correction\GeoCoDe\meta-llama-3.1-8b-instruct_with_meta-llama-3.1-8b-instruct_critic\20250122_seed_24_1000_articles\correct_articles_k_161.json", "r", encoding="utf-8") as f:
    correctly_geocoded_articles = json.load(f)
print(len(correctly_geocoded_articles))
correctly_geocoded_articles

154


[{'article_id': '10025588',
  'article_text': '110\xa0megawatt (MW) net[3] solar thermal power project with 1.1 gigawatt-hours of energy storage,[1] located near Tonopah, about 190 miles (310\xa0km) northwest of Las Vegas.[4][5]It substantially missed its intended power production by only achieving about 40% of its capacity on an annual basis, resulting in lawsuits and changes of control.[6][7] The site has not produced power since April 2019, and its sole customer has cancelled their contract.[8] While falling far short of its electrical capacity, Crescent Dunes was the first concentrated solar power (CSP) plant with a central receiver tower and advanced molten salt energy storage technology from SolarReserve.',
  'toponyms': ['Tonopah', 'Las Vegas'],
  'toponym_data': [{'osm_id': '170155',
    'coordinates': {'latitude': 38.0999749, 'longitude': -117.2506197},
    'toponym': 'Tonopah'},
   {'osm_id': '170117',
    'coordinates': {'latitude': 36.2551973, 'longitude': -115.2431753},
  

In [32]:
geo_relatable_harsh_from_geocoded_articles = [
    article for article in correctly_geocoded_articles
    if (any(word in article["article_text"] for word in ["lies", "located"]) and
        any(word in article["article_text"] for word in ["km", "kilometer", "kilometre"]))
        #and not "area" in article["article_text"])
]
print(len(geo_relatable_harsh_from_geocoded_articles))
print(f"# Toponyms: {sum(len(article['toponyms']) for article in geo_relatable_harsh_from_geocoded_articles)}")
geo_relatable_harsh_from_geocoded_articles

46
# Toponyms: 218


[{'article_id': '10025588',
  'article_text': '110\xa0megawatt (MW) net[3] solar thermal power project with 1.1 gigawatt-hours of energy storage,[1] located near Tonopah, about 190 miles (310\xa0km) northwest of Las Vegas.[4][5]It substantially missed its intended power production by only achieving about 40% of its capacity on an annual basis, resulting in lawsuits and changes of control.[6][7] The site has not produced power since April 2019, and its sole customer has cancelled their contract.[8] While falling far short of its electrical capacity, Crescent Dunes was the first concentrated solar power (CSP) plant with a central receiver tower and advanced molten salt energy storage technology from SolarReserve.',
  'toponyms': ['Tonopah', 'Las Vegas'],
  'toponym_data': [{'osm_id': '170155',
    'coordinates': {'latitude': 38.0999749, 'longitude': -117.2506197},
    'toponym': 'Tonopah'},
   {'osm_id': '170117',
    'coordinates': {'latitude': 36.2551973, 'longitude': -115.2431753},
  

# Run LLM GeoRelation for all fully correctly geocoded georelatable articles

In [33]:
with open(
        r"C:\Users\kaimo\Uni\Master\5_WS24_25-MA\geoparse-natural-disasters-with-llms\output\reflective_candidate_resolution\fatal_error_and_invalid_correction\GeoCoDe\meta-llama-3.1-8b-instruct_with_meta-llama-3.1-8b-instruct_critic\20250122_seed_24_1000_articles\articles_with_at_least_one_correct_toponym_k_161.json", "r", encoding="utf-8") as f:
    correctly_geocoded_articles = json.load(f)

In [34]:
def georelate(model_name, long_term_memory, article_text, mentioned_toponyms):
    prompt = long_term_memory.generate_georelation_prompt(article_text=article_text,
                                                          mentioned_toponyms=mentioned_toponyms,
                                                          example_path=r"/data/few_shot_example_georelation.json")
    handler = ChatAIHandler()
    model = handler.get_model(model_name)
    llm_answer = model.invoke(prompt)
    pattern = re.compile(r"<think>(.*?)</think>", re.DOTALL)

    thoughts = pattern.findall(llm_answer.content)  # Extract thoughts
    non_thoughts = pattern.sub("", llm_answer.content)  # Remove thoughts from text
    start = non_thoughts.find("```json")
    if start != -1:
        start = start + 7
        end = non_thoughts.find("```", start)  # Find the next occurrence after start
        content = non_thoughts[start:end]
        prediction = json.loads(content)
    elif non_thoughts.find("```") != -1:
        start = start + 4
        end = non_thoughts.find("```", start)  # Find the next occurrence after start
        content = non_thoughts[start:end]
        prediction = json.loads(content)
    else:
        prediction = json.loads(llm_answer.content)
    return thoughts, prediction

In [35]:
# model_name = "llama-3.3-70b-instruct" #"deepseek-r1-distill-llama-70b"
# directory = f'output/georelation/all_articles_with_at_lest_one_correctly_geocoded_toponym/{model_name}/{pd.Timestamp.now().strftime("%Y%m%d")}'
# output_directory = os.path.join(r"C:\Users\kaimo\Uni\Master\5_WS24_25-MA\geoparse-natural-disasters-with-llms",
#                                 directory)
# os.makedirs(output_directory, exist_ok=True)
#
# ltm = LongTermMemory(documentation_file=r"C:\Users\kaimo\Uni\Master\5_WS24_25-MA\geoparse-natural-disasters-with-llms\agent_components\memory\external_tool_documentation\geonames_websearch_documentation.md")
# handler = ChatAIHandler()
# model = handler.get_model(model_name)
# for article in correctly_geocoded_articles:
#     if os.path.exists(f"{output_directory}/{article['article_id']}.json"):
#         continue
#     prediction = georelate(model, ltm, article['article_text'], article['correctly_geocoded_toponyms'])
#     if prediction:
#         article.update({"georelation": prediction})
#         # save article to json file in a specified directory
#         try:
#             with open(f"{output_directory}/{article['article_id']}.json", "w", encoding="utf-8") as f:
#                 json.dump(article, f, ensure_ascii=False, indent=4)
#         except Exception as e:
#             article_ids = article["article_id"].split()
#             last_id = article_ids[-1]  # Get last ID
#             with open(f"{output_directory}/{last_id}.json", "w", encoding="utf-8") as f:
#                 json.dump(article, f, ensure_ascii=False, indent=4)

# Add ground truth center coordinates, bounding box and approx area to articles

In [36]:
def get_bounding_box(relation_id):
    query = f"""
    [out:json];
    relation({relation_id});
    out geom;
    """
    try:
        url = "https://overpass-api.de/api/interpreter"
        response = requests.post(url, data={"data": query})
        data = response.json()

        if not data['elements']:
            return None, None, None, None

        relation = data['elements'][0]
        if 'bounds' not in relation:
            return None, None, None, None

        bounds = relation['bounds']
        min_lat = bounds['minlat']
        max_lat = bounds['maxlat']
        min_lon = bounds['minlon']
        max_lon = bounds['maxlon']

        return min_lat, max_lat, min_lon, max_lon

    except Exception as e:
        print(f"Error fetching relation {relation_id}: {e}")
        return None, None, None, None

def get_osm_admin_center(osm_id):
    """
    Query Overpass API for the admin_center or center of the given OSM ID.
    """
    query = f"""
    [out:json];
    relation({osm_id});
    out center;
    """

    url = "https://overpass-api.de/api/interpreter"
    response = requests.get(url, params={"data": query})

    if response.status_code == 200:
        data = response.json()
        elements = data.get("elements", [])

        for element in elements:
            if "center" in element:
                return element["center"]  # Returns {lat, lon}

    return None  # Return None if no center/admin_center is found

def get_area_sq_km(geometry, source_crs="EPSG:4326", target_crs="EPSG:3857"):
    project = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True).transform
    projected_geometry = transform(project, geometry)
    area_sq_km = projected_geometry.area / 1_000_000
    return area_sq_km


def process_articles(articles):
    """
    Extracts the last ID from article_id and fetches its coordinates.
    """
    processed_articles = []
    results = {}

    for article in articles:
        article_ids = article["article_id"].split()
        last_id = article_ids[-1]  # Get last ID

        centroid_coords = get_osm_admin_center(last_id)

        bb_min_lat, bb_max_lat, bb_min_lon, bb_max_lon = get_bounding_box(last_id)
        if all((bb_min_lat, bb_max_lat, bb_min_lon, bb_max_lon)):
            bounding_box_polygon = Polygon([(bb_min_lon, bb_min_lat), (bb_min_lon, bb_max_lat),
                                        (bb_max_lon, bb_max_lat), (bb_max_lon, bb_min_lat)])

            bb_area = get_area_sq_km(bounding_box_polygon)
            if centroid_coords and bb_area:
                article.update({'gt': {'centroid': centroid_coords,
                                       'bounding_box':{
                                           'bb_min_lat': bb_min_lat,
                                           'bb_max_lat': bb_max_lat,
                                           'bb_min_lon': bb_min_lon,
                                           'bb_max_lon': bb_max_lon
                                       },
                                       'bb area': bb_area}})
                processed_articles.append(article)

    return processed_articles

In [302]:
# articles_for_georelation = process_articles(correctly_geocoded_articles)

In [38]:
with open(r"C:\Users\kaimo\Uni\Master\5_WS24_25-MA\geoparse-natural-disasters-with-llms\output\georelation\all_articles_with_at_lest_one_correctly_geocoded_toponym\articles_for_georelation.json", "r", encoding="utf-8") as f:
    articles_for_georelation = json.load(f)
len(articles_for_georelation)

510

# H3: Resolution handling

In [39]:
import pandas as pd
import numpy as np

# Define H3 resolutions with their corresponding hexagon areas (from table)
h3_data = [
    (0, 4357449.416078381),
    (1, 609788.441794133),
    (2, 86801.780398997),
    (3, 12393.434655088),
    (4, 1770.347654491),
    (5, 252.903858182),
    (6, 36.129062164),
    (7, 5.161293360),
    (8, 0.737327598),
    (9, 0.105332513),
    (10, 0.015047502),
    (11, 0.002149643),
    (12, 0.000307092),
    (13, 0.000043870),
    (14, 0.000006267),
    (15, 0.000000895),
]

# Convert to DataFrame
df = pd.DataFrame(h3_data, columns=["Resolution", "Hexagon_Area_km2"])

def get_h3_resolution_scaled(area_km2):
    """Find the H3 resolution where the input area and hexagon area are most proportionate."""
    df["Scale_Diff"] = np.abs(np.log2(area_km2 / df["Hexagon_Area_km2"]))
    closest_resolution = df.loc[df["Scale_Diff"].idxmin()]
    return int(closest_resolution["Resolution"])

# # Example usage:
# input_area = 100  # Change this to your area in km²
# best_resolution = get_h3_resolution_scaled(input_area)
# print(f"Best H3 resolution for {input_area} km²: {best_resolution}")

# GeoRelation Evaluation

In [41]:
from geopy.distance import geodesic

GEORELATION_DIR = r"C:\Users\kaimo\Uni\Master\5_WS24_25-MA\geoparse-natural-disasters-with-llms\output\georelation\all_articles_with_at_lest_one_correctly_geocoded_toponym\llama-3.3-70b-instruct\20250206"
k = 50

all_error_distances = []
all_squared_area_diffs = []
all_normalized_area_diffs = []
evaluation_results = []

for gt_article in articles_for_georelation:
    eval_results_for_article = {}
    article_ids = gt_article["article_id"].split()
    last_id = article_ids[-1]  # Get last ID

    # first, load generation
    try:
        generation_filepath = os.path.join(GEORELATION_DIR, f"{gt_article['article_id']}.json")
        with open(generation_filepath, "r", encoding="utf-8") as f:
            georelated_article = json.load(f)
    except Exception as e:
        generation_filepath = os.path.join(GEORELATION_DIR, f"{last_id}.json")
        with open(generation_filepath, "r", encoding="utf-8") as f:
            georelated_article = json.load(f)
    if not georelated_article:
        continue

    # calculate distance of coordinates
    gt_coordinates = (gt_article['gt']['centroid']['lat'],
                      gt_article['gt']['centroid']['lon'])
    predicted_coordinates = (georelated_article['georelation']['coordinates of geographical unit']['latitude'],
                             georelated_article['georelation']['coordinates of geographical unit']['longitude'])
    error_distance = geodesic(gt_coordinates, predicted_coordinates).kilometers
    all_error_distances.append(error_distance)

    # calculate area error distance
    gt_area = gt_article['gt']['bb area']
    predicted_area = georelated_article["georelation"]["area in square km"]

    squared_area_diff = (gt_area - predicted_area)**2
    all_squared_area_diffs.append(squared_area_diff)

    normalized_error_diff = np.abs((gt_area - predicted_area) / gt_area)
    all_normalized_area_diffs.append(normalized_error_diff)

    # calculate intersection
    # Define an H3 cell
    h3_index = h3.latlng_to_cell(
        lat=georelated_article["georelation"]["coordinates of geographical unit"]["latitude"],
        lng=georelated_article["georelation"]["coordinates of geographical unit"]["longitude"],
        res=get_h3_resolution_scaled(georelated_article["georelation"]["area in square km"])
    )

    # Get the boundary coordinates of the H3 cell
    hexagon = h3.cell_to_boundary(h3_index)

    # Convert polygon to shapely format
    hexagon_coords = [(lng, lat) for lat, lng in hexagon]  # Convert to (lng, lat) for Folium
    hexagon_polygon = Polygon(hexagon_coords)
    hexagon_area_estimate = get_area_sq_km(hexagon_polygon)

    bb = gt_article['gt']['bounding_box']
    bb_min_lat, bb_max_lat, bb_min_lon, bb_max_lon = bb['bb_min_lat'], bb['bb_max_lat'], bb['bb_min_lon'], bb['bb_max_lon']
    bounding_box_polygon = Polygon([(bb_min_lon, bb_min_lat), (bb_min_lon, bb_max_lat),
                                    (bb_max_lon, bb_max_lat), (bb_max_lon, bb_min_lat)])
    intersection = bounding_box_polygon.intersection(hexagon_polygon)
    if not intersection.is_empty:
        intersection_area = get_area_sq_km(intersection)
    else:
        intersection_area = 0

    eval_results_for_article.update({"article_id": last_id,
                                     "gt": gt_article['gt'],
                                     "georelation": georelated_article['georelation'],
                                     "absolute_error_distance": error_distance,
                                     "squared_area_error": squared_area_diff,
                                     "normalized_area_error": normalized_error_diff,
                                     "hexagon_area_estimate": hexagon_area_estimate,
                                     "area_intersection": intersection_area})
    evaluation_results.append(eval_results_for_article)

# Accuracy@k
within_k = [d <= k for d in all_error_distances]
accuracy_at_k = sum(within_k) / len(all_error_distances)
print(f"Accuracy@{k}: {accuracy_at_k}")

# AUC
def calculate_auc(sorted_values):
    max_error = 20039  # Earth's circumference in km / 2 (maximum possible distance)
    size = len(sorted_values)
    if size <= 1:
        return 0.0

    h = 1  # step size
    sum = 0.5 * (np.log(1 + sorted_values[0]) / np.log(max_error) + np.log(
        1 + sorted_values[-1]) / np.log(max_error))  # initial area

    for i in range(1, size - 1):
        sum += np.log(1 + sorted_values[i]) / np.log(max_error)

    auc = sum * h / (size - 1)
    return auc

sorted_error_distances = sorted(
    all_error_distances)  # assuming error_distances is a dictionary with error error_distances
auc = calculate_auc(sorted_error_distances)
print(f"AUC: {auc}")

# Mean error distance
mean_error_distance = np.mean(all_error_distances)
print(f"Mean error distance: {mean_error_distance}")

# Median error distance
median_error_distance = np.median(all_error_distances)
print(f"Median error distance: {median_error_distance}")

# RSME of area predictions
rsme = np.sqrt(np.sum(all_squared_area_diffs)/len(all_squared_area_diffs))
print(f"RSME of area predictions: {rsme}")

# Mean error of area predictions
area_errors = [np.sqrt(i) for i in all_squared_area_diffs]
mean_area_eror = np.mean(area_errors)
print(f"Mean area prediction error: {mean_area_eror}")
median_area_eror = np.median(area_errors)
print(f"Median area prediction error: {median_area_eror}")

# MAPE area predictions
mape = np.sum(all_normalized_area_diffs)/len(all_normalized_area_diffs)*100
print(f"MAPE area prediction: {mape}")

median_pe = np.median(all_normalized_area_diffs)
print(f"Median PE area prediction: {median_pe}")


C:\Users\kaimo\Uni\Master\5_WS24_25-MA\geoparse-natural-disasters-with-llms\venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


Accuracy@50: 0.9352941176470588
AUC: 0.2118605529563347
Mean error distance: 50.91792287932496
Median error distance: 6.1610711222515855
RSME of area predictions: 77499.53858921677
Mean area prediction error: 8509.975279771255
Median area prediction error: 64.53234647500611
MAPE area prediction: 1034.8488735521262
Median PE area prediction: 0.7702823526112743


In [42]:
average_precision = np.sum([evaluation["area_intersection"]/evaluation['hexagon_area_estimate'] for evaluation in evaluation_results])/len(evaluation_results)
average_recall = np.sum([evaluation["area_intersection"]/evaluation['gt']['bb area'] for evaluation in evaluation_results])/len(evaluation_results)
average_f1 = 2*average_precision*average_recall/(average_precision+average_recall)

print(f"average precision: {average_precision}, \naverage recall: {average_recall}, \naverage f1: {average_f1}")

average precision: 0.29626349443871486, 
average recall: 0.1886375498149302, 
average f1: 0.23050649344980562


In [44]:
def create_map(bb, hexagon, intersection):
    if not bb:
        return None
    lat_center, lon_center = (bb.centroid.y, bb.centroid.x)
    m = folium.Map(location=(lat_center, lon_center), zoom_start=10)

    folium.Polygon(locations=[(lat, lon) for lon, lat in bb.exterior.coords], color="yellow", fill=True, fill_opacity=0.3, weight=2).add_to(m)

    folium.Polygon(locations=[(lat, lon) for lon, lat in hexagon.exterior.coords], color="blue", fill=True, fill_opacity=0.3, weight=2).add_to(m)

    if intersection:
        folium.Polygon(locations=[(lat, lon) for lon, lat in intersection.exterior.coords],
                       color="green", fill=True, fill_opacity=0.3, weight=2).add_to(m)

    return m


ID = "4482873 5924711"
osm_id = 5924711
GEORELATION_DIR = r"C:\Users\kaimo\Uni\Master\5_WS24_25-MA\geoparse-natural-disasters-with-llms\output\georelation\all_articles_with_at_lest_one_correctly_geocoded_toponym\llama-3.3-70b-instruct\20250206"
generation_filepath = os.path.join(GEORELATION_DIR, f"{ID}.json")

with open(generation_filepath, "r", encoding="utf-8") as f:
    georelated_article = json.load(f)

georelation_prediction = georelated_article['georelation']

h3_index = h3.latlng_to_cell(
    georelation_prediction["coordinates of geographical unit"]["latitude"],
    georelation_prediction["coordinates of geographical unit"]["longitude"],
    get_h3_resolution_scaled(georelation_prediction["area in square km"])
)

hexagon = h3.cell_to_boundary(h3_index)

hexagon_coords = [(lon, lat) for lat, lon in hexagon]
hexagon_polygon = Polygon(hexagon_coords)
predicted_hexagon_area = get_area_sq_km(hexagon_polygon)
print(f"Predicted Hexagon area: {predicted_hexagon_area} square km")

bb_min_lat, bb_max_lat, bb_min_lon, bb_max_lon = get_bounding_box(osm_id)
if bb_min_lat is not None and bb_max_lat is not None and bb_min_lon is not None and bb_max_lon is not None:
    bounding_box_polygon = Polygon([(bb_min_lon, bb_min_lat), (bb_min_lon, bb_max_lat),
                                    (bb_max_lon, bb_max_lat), (bb_max_lon, bb_min_lat)])
    area = get_area_sq_km(bounding_box_polygon)
    print(f"Bounding box area: {area} square km")

    intersection = bounding_box_polygon.intersection(hexagon_polygon)

    if not intersection.is_empty:
        intersection_area = get_area_sq_km(intersection)
        print(f"Intersection area: {intersection_area} square km")

    m = create_map(bounding_box_polygon, hexagon_polygon, intersection)
else:
    print("Could not retrieve bounding box")

m

Predicted Hexagon area: 74.01845245208133 square km
Bounding box area: 114.13415555726272 square km
Intersection area: 28.511178245502464 square km


In [45]:
len(evaluation_results)

510

In [46]:
import stanza
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner') # initialize neural pipeline

C:\Users\kaimo\Uni\Master\5_WS24_25-MA\geoparse-natural-disasters-with-llms\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-18 18:02:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-02-18 18:02:39 INFO: Downloaded file to C:\Users\kaimo\stanza_resources\resources.json
2025-02-18 18:02:39 WARNING: Language en package default expects mwt, which has been added
2025-02-18 18:02:39 INFO: Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| ner       | ontonotes-ww-multi_charlm |

2025-02-18 18

In [47]:
article_text = "A fast-moving wildfire erupted in Southern California on Wednesday, destroying homes and sending firefighters rushing to get residents out of homes and to safety, officials said. \nThe Mountain Fire in Ventura County prompted evacuation orders and grew to over 14,000 acres, fueled by what fire officials called a significant Santa Ana wind event. \nFirefighters at the scene of the brush fire, which broke out between the communities of Moorpark and Somis, “were faced with a tough firefight,” Ventura County Fire Capt. Trevor Johnson said. \n“Firefighters were right off the bat engaged in pulling people out of their houses and saving lives,” Johnson said. \nThe fire was moving so fast that firefighters drove residents out of the area in fire engines because of the danger, he said. \nThe fire department did not have a count for the number of destroyed structures. The fire began at 8:51 a.m. local time, said the California Department of Forestry and Fire Protection, known as Cal Fire. It was 0% contained Wednesday afternoon, and what sparked it was under investigation."
article_title = "California wildfire fueled by high winds grows to over 14,000 acres and forces evacuations"

# article_text = "Die Erdbebenserie rund um die beliebte Ferieninsel Santorini dauert an. Die Menschen, die auf der Insel geblieben sind, wurden am Wochenende immer wieder in Angst versetzt und nachts aus dem Schlaf gerissen. Nach Angaben der Behörden hat die Erde seit dem 1. Februar Hunderte Male gebebt. Die Frequenz der Beben hat aber etwas nachgelassen. Die Seismologen geben aber keine Entwarnung. Es könnte doch noch ein größeres Beben geben, heißt es immer wieder seitens der Experten.\nSchulen bleiben geschlossen – viele Einwohner geflohen\nDie Regierung hat beschlossen, dass die Schulen auf Santorini und den benachbarten Eilanden Ios, Anafi und Amorgos auch kommende Woche geschlossen bleiben. \nDer größte Teil der Bevölkerung - vor allem Frauen, Kinder und ältere Menschen - hat die Insel verlassen und sich auf dem Festland in Sicherheit gebracht. \nAuf den Straßen trifft man Feuerwehrleute, Beamte des Zivilschutzes und der Polizei an, die aus anderen Regionen des Landes nach Santorini und den umliegenden Inseln gebracht worden sind - für den Fall, dass es zu einem schweren Erdbeben kommt. \nDie Stadt Athen hat zahlreiche Holzhäuser in einem Sommerlager für die Menschen geöffnet, die kein Geld oder Verwandte haben, um eine neue Bleibe zu finden, bis das Phänomen um Santorini endet. \nGefahr durch Vulkanaktivität?\nUnklar bleibt weiterhin, inwiefern die Erdbebenserie, die sich hauptsächlich zwischen den Inseln Santorini und Amorgos ereignet, die Vulkane der Region „geweckt“ haben könnte - eine Frage, die die Einwohner besorgt. \n1950 hatte eine Eruption des Vulkans von Santorini schwere Schäden angerichtet. Die Meinungen der Geologen und Seismologen gehen auseinander. Einige Experten schließen eine Eruption aus, andere halten sie für möglich.\nAusharrende geben sich gelassen\nViele Einwohner versuchen, sich Mut zu machen. Ein Mann aus Santorini sagte der Athener Zeitung „Kathimerini“ (Sonntag): „Santoriner zu sein bedeutet, auf einer Bombe zu leben und sich nicht darum zu kümmern.“"
# article_title = "Kein Ende der Beben um Santorini"
doc = nlp(article_text) # run annotation over a sentence
print(doc.entities)

[{
  "text": "Southern California",
  "type": "LOC",
  "start_char": 34,
  "end_char": 53
}, {
  "text": "Wednesday",
  "type": "DATE",
  "start_char": 57,
  "end_char": 66
}, {
  "text": "The Mountain Fire",
  "type": "FAC",
  "start_char": 180,
  "end_char": 197
}, {
  "text": "Ventura County",
  "type": "GPE",
  "start_char": 201,
  "end_char": 215
}, {
  "text": "over 14,000 acres",
  "type": "QUANTITY",
  "start_char": 255,
  "end_char": 272
}, {
  "text": "Santa Ana",
  "type": "GPE",
  "start_char": 325,
  "end_char": 334
}, {
  "text": "Moorpark",
  "type": "GPE",
  "start_char": 436,
  "end_char": 444
}, {
  "text": "Somis",
  "type": "GPE",
  "start_char": 449,
  "end_char": 454
}, {
  "text": "Ventura County",
  "type": "GPE",
  "start_char": 493,
  "end_char": 507
}, {
  "text": "Trevor Johnson",
  "type": "PERSON",
  "start_char": 519,
  "end_char": 533
}, {
  "text": "Johnson",
  "type": "PERSON",
  "start_char": 643,
  "end_char": 650
}, {
  "text": "8:51 a.m.",
  "type"

In [48]:
toponym_list = [entity.text for entity in doc.entities if entity.type in ["LOC", "GPE"]]

In [96]:
"""
Initialize GeoCoder
"""
from helpers.helpers import preprocess_data
from models.candidates import CandidateGenerationState, GeoCodingState
import time
from modules.reflective_geocoding import ReflectiveGeoCoder

call_times = []
actor = "meta-llama-3.1-8b-instruct"  # ["deepseek-r1-distill-llama-70b", "meta-llama-3.1-8b-instruct", "llama-3.3-70b-instruct", "mistral-large-instruct"]
critic = "mistral-large-instruct"
dataset = "New"

geocoder = ReflectiveGeoCoder(
    actor_model_name=actor,
    critic_model_name=critic,
    call_times=call_times,
    skip_few_shot_loader=False,
    data_set=dataset
)

"""
Reflective Candidate Generation
"""
generation_graph_builder = geocoder.build_graph()
generation_agent_graph = generation_graph_builder.compile()
start = time.time()
input_state = {
        "article_id": "002",
        "article_title": article_title,
        "article_text": article_text,
        "toponyms": toponym_list
}

# Invoke the graph
generation_agent_graph_answer = generation_agent_graph.invoke(input_state)
candidate_generation = CandidateGenerationState(**generation_agent_graph_answer)
print(f"generation with {actor}_with_{critic}_critic : time taken: {time.time() - start} seconds.")

generation with meta-llama-3.1-8b-instruct_with_mistral-large-instruct_critic : time taken: 5.192487001419067 seconds.


In [98]:
"""
Run reflective candidate resolution graph
"""
candidate_generation_dict = preprocess_data(candidate_generation.model_dump(), GeoCodingState)
resolution_graph_builder = geocoder.build_resolution_graph()
resolution_agent_graph = resolution_graph_builder.compile()
start = time.time()
resolution_agent_graph_answer = resolution_agent_graph.invoke(candidate_generation_dict)
candidate_resolution = GeoCodingState(**resolution_agent_graph_answer)
print(f"resolution with {actor}_with_{critic}_critic for: time taken: {time.time() - start} seconds.\n")

resolution with meta-llama-3.1-8b-instruct_with_mistral-large-instruct_critic for: time taken: 6.9005303382873535 seconds.



In [49]:
"""
GeoRelation
"""
from models.candidates import GeoCodingState
with open(r"C:\Users\kaimo\Uni\Master\5_WS24_25-MA\geoparse-natural-disasters-with-llms\output\georelation\the_mountain_fire_geocoding.json", "r", encoding="utf-8") as f:
    candidate_resolution = json.load(f)
    candidate_resolution = GeoCodingState(**candidate_resolution)
geocoded_toponyms = []
for topo in candidate_resolution.valid_geocoded_toponyms:
    for item in candidate_resolution.toponyms_with_candidates:
        if item.toponym_with_search_arguments.toponym.casefold() == topo.toponym.casefold():
            # get the candidate with the correct geonameid
            incorrect_geonameid = True
            for candidate in item.candidates:
                if topo.selected_candidate_geonameId == candidate["geonameId"]:
                    topo.coordinates = {"latitude": candidate["lat"],
                                        "longitude": candidate["lng"]}
                    geocoded_toponyms.append(topo)
                    break
            break
geocoded_toponyms = [toponym.model_dump() for toponym in geocoded_toponyms]

In [107]:
# thoughts, prediction = georelate("deepseek-r1-distill-llama-70b", geocoder.working_memory.long_term_memory, article_text, geocoded_toponyms)

In [53]:
with open(r"C:\Users\kaimo\Uni\Master\5_WS24_25-MA\geoparse-natural-disasters-with-llms\output\georelation\the_mountain_fire_georelation_llama_distill_R1.json", "r", encoding="utf-8") as f:
    output = json.load(f)
thoughts = output['thoughts']
prediction = output['prediction']
print(thoughts)
print(json.dumps(prediction, indent = 4))


Alright, I'm trying to figure out how to respond to this query. The user has provided an article about a wildfire in Southern California, specifically in Ventura County. They've also given me a list of mentioned toponyms with their coordinates. My task is to determine the coordinates of the main geographical unit discussed in the article and estimate its area.

First, I need to understand the structure of the problem. The article is about the Mountain Fire in Ventura County. The main geographical unit here is the fire itself, but since fires don't have fixed coordinates, I think the user is asking for the origin point or the general area where the fire occurred.

Looking at the mentioned toponyms, I see Southern California, Ventura County, Santa Ana, Moorpark, Somis, and Ventura County again. The coordinates for these places are provided. The fire broke out between Moorpark and Somis, so the location should be somewhere in that vicinity.

I should check the coordinates for Moorpark an

In [56]:
h3_res = get_h3_resolution_scaled(prediction["area in square km"])
h3_index = h3.latlng_to_cell(
    prediction["coordinates of geographical unit"]["latitude"],
    prediction["coordinates of geographical unit"]["longitude"],
    h3_res
)

hexagon = h3.cell_to_boundary(h3_index)

hexagon_coords = [(lon, lat) for lat, lon in hexagon]
hexagon_polygon = Polygon(hexagon_coords)
predicted_hexagon_area = get_area_sq_km(hexagon_polygon)
lat_center, lon_center = (prediction["coordinates of geographical unit"]["latitude"], prediction["coordinates of geographical unit"]["longitude"])
m = folium.Map(location=(lat_center, lon_center), zoom_start=10)
folium.Polygon(locations=[(lat, lon) for lon, lat in hexagon_polygon.exterior.coords], color="red", fill=True, fill_opacity=0.3, weight=2).add_to(m)
for loc in geocoded_toponyms:
    folium.Marker(
        location=[loc['coordinates']['latitude'], loc['coordinates']['longitude']],
        popup=loc['toponym'],
        tooltip=loc['toponym']
    ).add_to(m)
m.save("the_mountain_fire_georelated.html")
m

In [14]:
import urllib.parse
import requests
import os
import folium
import json

base_url = "http://api.geonames.org/search?"

params = {"name_equals": "Moorpark",
          "username": "KaiMo",
          "type": "json",
          "style": "full"}
url = base_url + urllib.parse.urlencode(params)
response = requests.get(url)
if response.status_code != 200:
    response = requests.get(url) #retry once
    if response.status_code != 200:
        raise Exception(f"Error in GeoNamesAPI.search: {response.text}")
all_moorparks = response.json()
moorparks_map = folium.Map()
for loc in all_moorparks['geonames']:
    folium.Marker(
        location=[loc['lat'], loc['lng']],
        popup=loc['name'],
        tooltip=loc['name']
    ).add_to(moorparks_map)
moorparks_map.save("all_moorparks_map.html")
print(json.dumps(all_moorparks, indent=4))
moorparks_map

{
    "totalResultsCount": 3,
    "geonames": [
        {
            "timezone": {
                "gmtOffset": -8,
                "timeZoneId": "America/Los_Angeles",
                "dstOffset": -7
            },
            "bbox": {
                "east": -118.82076700000002,
                "south": 34.259003,
                "north": 34.30791,
                "west": -118.92083099999998,
                "accuracyLevel": 10
            },
            "asciiName": "Moorpark",
            "astergdem": 152,
            "countryId": "6252001",
            "fcl": "P",
            "srtm3": 160,
            "score": 135.74127197265625,
            "adminId2": "5405889",
            "countryCode": "US",
            "adminCodes1": {
                "ISO3166_2": "CA"
            },
            "adminId1": "5332921",
            "lat": "34.28556",
            "fcode": "PPL",
            "continentCode": "NA",
            "elevation": 157,
            "adminCode2": "111",
            "admi